In [1]:
using Symbolics
using LinearAlgebra
using HDF5

function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

b⁰ = [0, 0, 1, 1]
b¹ = [0, 0, -1, 1]
bˣ = [1, 0, 0, 1]
bʸ = [1, 1, 0, 1]

function read_Ac(models_file_name, γ)
    h5open(models_file_name, "r") do file
        Ac = read(file["gamma_"*string(γ)]["Ac"])
        return Ac
    end
end


read_Ac (generic function with 1 method)

In [2]:
# Define symbolic variables
@variables ω α β

@variables γ δ γ₀ δ₀ γ₁ δ₁

Hᴸᴹᴱ = [ ω             α+im*β
         α-im*β        0      ]


Jᴸᴹᴱ = [ γ₀+im*δ₀     γ+im*δ
         γ₁+im*δ₁    -γ₀-im*δ₀  ]


function LME_operator_symbolic(H, J)

    Id = I(2)   
    U = -im*(Id ⊗ H - transpose(H) ⊗ Id)
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 

    Lᴸᴹᴱ = U + D

    return Lᴸᴹᴱ
end

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

4×4 Matrix{Complex{Num}}:
                                                                         (1//2)*(-2(γ₁^2) - 2(δ₁^2))  …                                                                           (1//2)*(2(γ^2) + 2(δ^2))
 (1//2)*(3γ₀*γ₁ + 3δ₀*δ₁ - γ*γ₀ - δ*δ₀) + im*((1//2)*(γ₀*δ + 3γ₀*δ₁ - γ*δ₀ - 3γ₁*δ₀) - α) - β                 β + (1//2)*(γ₀*γ₁ + δ₀*δ₁ - 3γ*γ₀ - 3δ*δ₀) + im*(α + (1//2)*(γ₀*δ₁ + 3γ₀*δ - γ₁*δ₀ - 3γ*δ₀))
 (1//2)*(3γ₀*γ₁ + 3δ₀*δ₁ - γ*γ₀ - δ*δ₀) + im*(α + (1//2)*(γ*δ₀ + 3γ₁*δ₀ - γ₀*δ - 3γ₀*δ₁)) - β            β + (1//2)*(γ₀*γ₁ + δ₀*δ₁ - 3γ*γ₀ - 3δ*δ₀) + im*((1//2)*(γ₁*δ₀ + 3γ*δ₀ - 3γ₀*δ - γ₀*δ₁) - α)
                                                                          (1//2)*(2(γ₁^2) + 2(δ₁^2))                                                                             (1//2)*(-2(γ^2) - 2(δ^2))

In [3]:
M =   [ 0  0    1  1
        1  -im  0  0 
        1  im   0  0
        0  0   -1  1 ]/2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)

4×4 Matrix{Num}:
 (1//2)*(2γ*γ₁ + 2δ*δ₁) + (1//2)*(-(γ^2) - (γ₁^2) - (δ^2) - 4(γ₀^2) - 4(δ₀^2) - (δ₁^2))                                                               …  0.5(γ₀*γ₁ + δ₀*δ₁ - 3γ*γ₀ - 3δ*δ₀ + 2β) + 0.5(3γ₀*γ₁ + 3δ₀*δ₁ - γ*γ₀ - δ*δ₀ - 2β)
                                                                                      0.5(-ω - (1//2)*(2γ*δ₁ - 2γ₁*δ)) + 0.5(0.5(2γ₁*δ - 2γ*δ₁) - ω)     0.5(0.5(γ*δ₀ + 3γ₁*δ₀ - γ₀*δ - 3γ₀*δ₁) + 2.0α - (1//2)*(γ₀*δ + 3γ₀*δ₁ - γ*δ₀ - 3γ₁*δ₀)) + 0.5(0.5(γ₁*δ₀ + 3γ*δ₀ - 3γ₀*δ - γ₀*δ₁) - (1//2)*(γ₀*δ₁ + 3γ₀*δ - γ₁*δ₀ - 3γ*δ₀) - 2.0α)
                                               (1//2)*(γ*γ₀ + γ₀*γ₁ + δ*δ₀ + δ₀*δ₁) + 2.0β - 0.5(-γ*γ₀ - γ₀*γ₁ - δ*δ₀ - δ₀*δ₁)                           0.5((1//2)*(2(γ^2) + 2(δ^2)) - 0.5(-2(γ^2) - 2(δ^2))) + 0.5((1//2)*(-2(γ₁^2) - 2(δ₁^2)) - 0.5(2(γ₁^2) + 2(δ₁^2)))
           (1//2)*(γ*γ₀ + γ₀*γ₁ + δ*δ₀ + δ₀*δ₁) + (1//2)*(-γ*γ₀ - γ₀*γ₁ - δ*δ₀ - δ₀*δ₁)                                                               

In [4]:
function SumCoeffs2(poly, s)
    sum_of_coeffs2 = 0
    expanded_poly = Symbolics.expand(poly)
    # Assuming the polynomial is a sum of terms
    for term in expanded_poly.args
    # Extract the coefficient from each term
        if isa(term, SymbolicUtils.Add)
            # If the term itself is a sum, handle accordingly
            for subterm in term.args
                coeff = Symbolics.coefficient(subterm, s, 0)
                sum_of_coeffs += coeff
            end
        else
            coeff = Symbolics.coefficient(term, s, 0)
            sum_of_coeffs2 += coeff^2
        end
    end

    return(sum_of_coeffs2)

end


function TF_objective(Aˢʸᵐᵇ, Aˢⁱᵈ, b)

    @variables s

    Gˢʸᵐᵇ = inv(I(4)*s - Aˢʸᵐᵇ)*b

    Gˢⁱᵈ = inv(I(4)*s - Aˢⁱᵈ)*b

    #ΔG = SymPy.together.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3])
    ΔG = simplify.(Gˢʸᵐᵇ - Gˢⁱᵈ)

    polys = [Δg.args[1] for Δg in ΔG]

    obj = sum(SumCoeffs2(polys.^2, s))

    #objective = sympy_to_dynamicpoly(obj)

    return obj

end

function sb_TF_objective(γ, Acˢʸᵐᵇ)
    models_file_name="../MODELS/DMD_b4_LTI_trn4_2023-Nov-29_at_17-38.h5"
    Acᴰᴹᴰ = read_Ac(models_file_name, γ)
    obj =  TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, bˣ) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, bʸ) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, b¹) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, b⁰)
    objective = sympy_to_dynamicpoly(obj)
    return objective
end


sb_TF_objective (generic function with 1 method)

In [5]:
models_file_name="../MODELS/DMD_b4_LTI_trn4_2023-Nov-29_at_17-38.h5"
Acᴰᴹᴰ = read_Ac(models_file_name, "0.079477")

Aˢⁱᵈ = Acᴰᴹᴰ

4×4 Matrix{Float64}:
 -3.25665e-7   -25.1327        1.40648e-7   1.43238e-8
 25.1379        -0.0796273     1.09697e-5   2.46497e-6
 -0.000892152   -0.000126719  -0.0794572    0.0794907
  1.3011e-14    -3.22023e-14   1.70138e-14  3.84902e-14

In [6]:
Aˢʸᵐᵇ = Aᴸᴹᴱ

@variables s

Gˢʸᵐᵇ = inv(I(4)*s - Aˢʸᵐᵇ)*bˣ

4-element Vector{Num}:
 (-(-0.5(γ₀*γ₁ + δ₀*δ₁ - 3γ*γ₀ - 3δ*δ₀ + 2β) - 0.5(3γ₀*γ₁ + 3δ₀*δ₁ - γ*γ₀ - δ*δ₀ - 2β))*((s + 0.5((1//2)*(2(γ^2) + 2(δ^2)) - 0.5(-2(γ^2) - 2(δ^2))) - 0.5((1//2)*(-2(γ₁^2) - 2(δ₁^2)) - 0.5(2(γ₁^2) + 2(δ₁^2))))*(s + 0.5((1//2)*(2γ*γ₁ + 2δ*δ₁) - 0.5(-(γ^2) - (γ₁^2) - (δ^2) - 4(γ₀^2) - 4(δ₀^2) - (δ₁^2))) - 0.5((1//2)*(-(γ^2) - (γ₁^2) - (δ^2) - 4(γ₀^2) - 4(δ₀^2) - (δ₁^2)) - 0.5(2γ*γ₁ + 2δ*δ₁))) - (0.5(0.5(γ₁*δ₀ + 3γ*δ₀ - 3γ₀*δ - γ₀*δ₁) - (1//2)*(γ₀*δ₁ + 3γ₀*δ - γ₁*δ₀ - 3γ*δ₀) - 2.0α) - 0.5(0.5(γ*δ₀ + 3γ₁*δ₀ - γ₀*δ - 3γ₀*δ₁) + 2.0α - (1//2)*(γ₀*δ + 3γ₀*δ₁ - γ*δ₀ - 3γ₁*δ₀)))*(0.5(2.0α + (1//2)*(γ*δ₀ + γ₀*δ₁ - γ₀*δ - γ₁*δ₀) - 0.5(γ₀*δ + γ₁*δ₀ - γ*δ₀ - γ₀*δ₁)) - 0.5((1//2)*(γ₀*δ + γ₁*δ₀ - γ*δ₀ - γ₀*δ₁) - 2.0α - 0.5(γ*δ₀ + γ₀*δ₁ - γ₀*δ - γ₁*δ₀)))) - (0.5((1//2)*(2γ*δ₁ - 2γ₁*δ) - ω) - 0.5(ω + (1//2)*(2γ₁*δ - 2γ*δ₁)))*((-0.5((1//2)*(2(γ^2) + 2(δ^2)) - 0.5(-2(γ^2) - 2(δ^2))) - 0.5((1//2)*(-2(γ₁^2) - 2(δ₁^2)) - 0.5(2(γ₁^2) + 2(δ₁^2))))*(0.5(0.5(γ₁*δ₀ + 3γ*δ₀ - 3γ₀*δ - γ₀*δ₁) -

In [7]:
Gˢⁱᵈ = inv(I(4)*s - Aˢⁱᵈ)*bˣ

4-element Vector{Num}:
   (1.4323798596880449e-8(1.3900711761257261e-9 + (0.07945724160052216 + s)*(0.07962729399778215 + s)) - 1.4064814856320953e-7(3.1235922867463774e-10 - 0.07949065709079117(0.07962729399778215 + s)) - 25.132748097663804(8.719892195015873e-7 + 2.4649739408523384e-6(0.07945724160052216 + s))) / ((3.2566493068499746e-7 + s)*(1.0969707024885135e-5(2.5597845823694687e-15 + 0.00012671907945875898(s - 3.849019740660616e-14)) + (0.07962729399778215 + s)*((s - 3.849019740660616e-14)*(0.07945724160052216 + s) - 1.3524386006844376e-15) - 2.4649739408523384e-6(-2.155973805921376e-18 - 3.220233265207232e-14(0.07945724160052216 + s))) + 1.4323798596880449e-8((1.301104392522212e-14(0.07945724160052216 + s) - 1.5178907658922002e-17)*(-0.07962729399778215 - s) - 3.3323925052945895e-22 - 25.137881395636413(-2.155973805921376e-18 - 3.220233265207232e-14(0.07945724160052216 + s))) - 1.4064814856320953e-7((-0.07962729399778215 - s)*(0.000892152400203255(s - 3.849019740660616e-14) - 1.

In [9]:
import Pkg; Pkg.add("SymbolicUtils")

   Resolving package versions...


    Updating `~/github/OQSID-thesis/Project.toml`
⌃ [d1185830] + SymbolicUtils v1.1.0
  No Changes to `~/github/OQSID-thesis/Manifest.toml`


In [10]:
using SymbolicUtils

In [11]:
simplify_fractions.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3])

In [9]:
ΔG = simplify.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3], expand=true, simplify_fractions=true)

In [8]:
#ΔG = SymPy.together.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3])
ΔG = simplify.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3], expand=true, simplify_fractions=false)



3-element Vector{Num}:
  (2.6837005316612064e-5 + 6.193811021415177e-5s - 1.4323798596880449e-8(s^2)) / (s^4 + 50.19981692775302s + 0.1590848612631965(s^3) + 631.7903678439403(s^2) - 2.786649629259899e-12) + (3.511828241382704e-16 - 0.006326966527241284s - 0.1590845355982658(s^2) - (s^3)) / (s^4 + 50.19981692775302s + 0.1590848612631965(s^3) + 631.7903678439403(s^2) - 2.786649629259899e-12) + (s^3 + 2.0(s^2)*(γ₀^2 + δ₀^2) + 0.5s*(γ^4 + γ₁^4 + δ^4 + δ₁^4) + γ*γ₁*(s^2) + δ*δ₁*(s^2) + 4.0s*(α^2) + s*(γ₀^2)*(δ^2 + δ₁^2) + s*(γ^2 + γ₁^2)*(δ^2 + δ₀^2 + δ₁^2) + s*γ*(γ₁^3) + s*γ₁*(γ^3) + 1.5(γ^2 + γ₁^2 + δ^2 + δ₁^2)*(s^2) + s*(γ^2)*(γ₁^2) + s*δ*(δ₁^3) + s*δ₁*(δ^3) + s*(δ^2)*(δ₁^2) + 2.0s*(γ^2)*(γ₀^2) + s*γ*γ₁*(δ^2 + δ₁^2) + s*δ*δ₁*(γ^2 + γ₁^2) + 2.0s*(γ₀^2)*(γ₁^2) + 2.0s*(δ^2)*(δ₀^2) + 2.0s*(δ₀^2)*(δ₁^2) + 2.0s*γ*γ₁*(δ₀^2) + 2.0s*δ*δ₁*(γ₀^2) + 2.0s*γ*γ₀*δ*δ₀ + 2.0s*γ₀*γ₁*δ₀*δ₁ - 2.0s*γ₀*γ₁*δ*δ₀ - 2.0s*γ*γ₀*δ₀*δ₁) / (s^4 + (s^2)*(ω^2) + 0.25s*(γ^6 + γ₁^6 + δ^6 + δ₁^6) + 2.0(γ^2 + γ₁^2 + δ₁^2)*(

In [20]:
γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

In [26]:
sb_TF_objective( γ_list[1] , Aᴸᴹᴱ)

In [ ]:
TF_objective(Aˢʸᵐᵇ, Aˢⁱᵈ, b)

In [2]:
using Symbolics

# Define symbolic variables
@variables x y

# Create a symbolic matrix
A = [x y; y x^2]

# Invert the matrix symbolically
A_inv = Symbolics.inv(A)

# A_inv now holds the inverted matrix

2×2 Matrix{Num}:
 (x^2) / (x^3 - (y^2))  (-y) / (x^3 - (y^2))
  (-y) / (x^3 - (y^2))     x / (x^3 - (y^2))